In [9]:
import pandas as pd
import numpy as np

In [30]:
test_df = pd.read_parquet('test.parquet')

In [31]:
test_df['ts_start_date'] = test_df['dates'].apply(lambda x: x[0])
test_df['mean'] = test_df['values'].apply(lambda x: x.mean())
test_df['var'] = test_df['values'].apply(lambda x: x.var())
test_df['max'] = test_df['values'].apply(lambda x: x.max())

fft = test_df['values'].apply(lambda x: np.fft.fft(x))

for i in range(5):
    test_df[f'f_coeff_{i}_real'] = fft.apply(lambda x: x[i].real)
    test_df[f'f_coeff_{i}_imag'] = fft.apply(lambda x: x[i].imag)
    test_df[f'f_coeff_{i}_angle'] = fft.apply(lambda x: np.angle(x[i]))


#### Загрузка модели

In [32]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(cat_features=['ts_start_date'])
clf.load_model("trained_model")

In [51]:
exclude_features = ['id', 'values', 'dates', 'f_coeff_0_real']

cat_features = ['ts_start_date']



submission = pd.read_csv('sample_submission.csv')

final_df = submission[['id']].merge(test_df, on="id", how="left").drop(exclude_features, axis=1)
final_df['ts_start_date'] = final_df['ts_start_date'].astype(str)

#### Формируем файл с предсказаниями

In [52]:
submission['score'] = clf.predict_proba(final_df)[:, 1]
submission.to_csv('submission.csv', index=False)